# Clusterização (Modelo Não Supervisionado DBSCAN)

Este notebook explora o algoritmo de aprendizado de máquina não supervisionado DBSCAN (Density-Based Spatial Clustering of Applications with Noise). <br> O objetivo deste método é identificar clusters de alta densidade em um conjunto de dados, enquanto separa os pontos que são considerados ruído. Através dessa abordagem, o DBSCAN é capaz de detectar formas arbitrárias de clusters, permitindo uma análise mais robusta das estruturas subjacentes nos dados.</p>


---
## Instalação de bibliotecas
Instala bibliotecas necessárias:

In [ ]:
!pip install pandas==2.1.4 numpy==1.26.4 matplotlib==3.7.5 seaborn==0.13.2 scikit-learn==1.4.2 plotly==5.24.1

---
## Inicialização de bibliotecas
Importa as bibliotecas necessárias:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import plotly.graph_objs as go

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import IsolationForest
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score, silhouette_samples
from matplotlib.ticker import PercentFormatter

pd.set_option('display.max_columns', None)  # Mostra todas as colunas

# Configuração para exibir os gráficos diretamente no notebook
%matplotlib inline

---
## Importe do banco de dados via arquivo local
Importação do DataFrame da base de dados da Unipar com a biblioteca pandas:

Para rodar o notebook corretamente, será necessário importar o arquivo de banco de dados manualmente, pois ele não está incluído no repositório devido à presença de dados sensíveis.

#### Passos para Importar o Banco de Dados no VS Code

1. **Obtenha o Arquivo de Dados**:
   - O arquivo `BASE DE SINISTRO UNIPAR BRADESCO.csv` deverá ser fornecido separadamente. Entre em contato com o responsável pelo projeto para receber o arquivo.

2. **Posicione o Arquivo na Pasta Correta**:
   - Após receber o arquivo, arraste-o para a mesma pasta onde o notebook está localizado em seu computador. Isso garante que o caminho relativo na função de leitura permaneça o mesmo e funcione corretamente.
   
3. **Verifique o Caminho do Arquivo**:
   - O código de leitura do arquivo já está implementado no notebook e não precisa ser alterado:
     ```python
     df = pd.read_csv('BASE DE SINISTRO UNIPAR BRADESCO.csv', decimal=',')
     ```
   - Certifique-se de que o arquivo CSV esteja no mesmo diretório que o notebook para evitar problemas de caminho.

4. **Rodar o Notebook**:
   - Com o arquivo posicionado corretamente, execute o notebook normalmente. O pandas irá carregar os dados e você poderá seguir com a análise.

**Nota**: Caso o arquivo não esteja na mesma pasta que o notebook, o código não será capaz de localizar o banco de dados, resultando em um erro. Portanto, é essencial que o arquivo seja arrastado para o diretório correto antes da execução.

&ensp;Agora que você baixou os notebooks e o banco de dados, pode seguir para os próximos passos de execução, seja localmente ou no Google Colab.

In [3]:
df = pd.read_csv('BASE DE SINISTRO UNIPAR BRADESCO.csv', decimal=',')

---
## Pré-Processamento
Como apresentado no notebook 'Pré-Processamento', as linhas a seguir executam várias etapas para limpar e organizar o banco de dados.<br>
Os comentários no código explicam cada ação.

In [ ]:
# Limpeza de dados

# Tratamento de valores nulos
df = df.dropna()
# Correção do ponto faltante em 'UNIPAR INDUPA DO BRASIL S.A'
df = df.replace({"UNIPAR INDUPA DO BRASIL S.A": "UNIPAR INDUPA DO BRASIL S.A."})
# Remoção de AGREGADO e DEPENDENTE
df_remove_d = df.loc[(df['Elegibilidade Sinistro'] == 'DEPENDENTE') ]
df = df.drop(df_remove_d.index)
df_remove_a = df.loc[(df['Elegibilidade Sinistro'] == 'AGREGADO') ]
df = df.drop(df_remove_a.index)
# Tratamento de valores duplicados
df = df.drop_duplicates(keep='last')

df

---
## Codificação
Este trecho de código realiza a codificação de colunas categóricas, transformando-as em valores numéricos para facilitar o uso no modelo<br>
de machine learning. As colunas categóricas são mapeadas para índices numéricos, e o resultado é armazenado em um novo DataFrame.<br>
Além disso, a coluna de data é convertida para o formato 'YYYYMMDD', e as colunas numéricas e de data são adicionadas ao DataFrame final.

In [5]:
# Lista de colunas que serão codificadas (colunas categóricas)
colunas_para_codificar = [
    'Codigo Empresa Sinistro',
    'Sexo Sinistro',
    'Faixa-Etária Nova Sinistro',
    'Descricao Plano Sinistro',
    'Codigo Servico Sinistro',
]

# Colunas numéricas e de data que não precisam de codificação
colunas_nao_codificadas = [
    'Dt Data Sinistro',
    'Valor Pago Sinistro',
]

# Para armazenar as novas colunas codificadas
novas_colunas_codificadas = {}

# Codificação das colunas categóricas
for coluna in colunas_para_codificar:
        unique_sorted_values = sorted(df[coluna].unique())
        df[f'{coluna}'] = df[coluna].apply(lambda x: unique_sorted_values.index(x))
        # Armazenar as novas colunas codificadas
        novas_colunas_codificadas[f'{coluna}'] = df[f'{coluna}']

# Criando um novo DataFrame com as novas colunas codificadas
df_novo = pd.DataFrame(novas_colunas_codificadas)

# Convertendo a coluna 'Dt Data Sinistro' para o formato 'YYYYMMDD' (Ano-Mês-Dia)
df['Dt Data Sinistro'] = pd.to_datetime(df['Dt Data Sinistro'], format='%d/%m/%Y').dt.strftime('%Y%m%d')

# Adicionando as colunas numéricas e de data ao DataFrame final
for coluna in colunas_nao_codificadas:
    df_novo[coluna] = df[coluna]

df = df_novo

#### Verificação

In [ ]:
# Exibindo as 10 primeiras linhas do novo DataFrame
df.head(10).sort_values(by='Valor Pago Sinistro', ascending=False)

---
## Detecção e Remoção de Outliers
Esse bloco de código remove os outlier da coluna *Valor Pago Sinistro* aplicando o _IsolationForest_ com uma contaminação de 5%, criando um novo DataFrame chamado de `df_clean`. Além disso, ele padroniza as colunas numéricas e as armazena em uma nova variável `numeric_colums` apenas com as colunas numéricas do DataFrame para futuras análises.


In [7]:
# Aplicar IsolationForest apenas na coluna 'Valor Pago Sinistro'
iso_forest = IsolationForest(contamination=0.05)  # Ajuste a contaminação conforme necessário
outliers = iso_forest.fit_predict(df[['Valor Pago Sinistro']])

# Remover os outliers apenas da coluna 'Valor Pago Sinistro'
df_clean = df[outliers == 1]

# Selecionar apenas as colunas numéricas para padronização
numeric_columns = df_clean.select_dtypes(include=['float64', 'int64']).columns


#### Verificação

In [ ]:
df_clean.describe().round(2) #Arredonda para duas casas decimais

---
## Tratamento de Dados

Este código padroniza as colunas numéricas no DataFrame `df_clean` após a remoção de outliers, utilizando o `StandardScaler` para ajustar os dados à média 0 e desvio padrão 1. 

A função `fit_transform` é aplicada para transformar as colunas numéricas, e o resultado é armazenado diretamente no DataFrame. Por fim, as primeiras linhas do DataFrame limpo e padronizado são exibidas com `df_clean.head()`.


In [ ]:
# Padronizar os dados após remover os outliers
scaler = StandardScaler()
df_clean.loc[:, numeric_columns] = scaler.fit_transform(df_clean[numeric_columns])  # Usando .loc para evitar o SettingWithCopyWarning

# Mostrar as primeiras linhas do dataframe limpo e padronizado
df_clean.head()

#### Verificação

In [ ]:
df.head(10).sort_values(by='Valor Pago Sinistro', ascending=False)

In [ ]:
df.max()

---
## Conversão e Extração de Componentes de Data

Este código converte a coluna `Dt Data Sinistro` do DataFrame `df` para o formato datetime, utilizando o padrão 'YYYYMMDD'. Em seguida, extrai o componente do mês da data e o armazena em uma nova coluna chamada `Mes`. Se necessário, os valores dessa coluna são padronizados utilizando o `StandardScaler`, com o resultado armazenado na nova coluna `Mes_scaled`. O DataFrame resultante contém as colunas originais e as novas colunas derivadas da data.


In [ ]:
# Converter a coluna para datetime
df['Dt Data Sinistro'] = pd.to_datetime(df['Dt Data Sinistro'], format='%Y%m%d')

# Extrair componentes de dia
df['Mes'] = df['Dt Data Sinistro'].dt.month

# Padronizar (escalonar) os componentes se necessário
scaler = StandardScaler()

df[["Mes_scaled"]] = scaler.fit_transform(df[["Mes"]])

df.head()

---
## Remoção de Colunas
Este código remove as colunas `Mes` e `Dt Data Sinistro` do DataFrame `df`. A remoção dessas colunas é feita diretamente no DataFrame, utilizando o parâmetro `inplace=True`, o que garante que as alterações sejam aplicadas sem a necessidade de criar uma cópia do DataFrame. O resultado é um DataFrame que não contém mais as colunas especificadas.


In [13]:
df.drop(columns=['Mes'], inplace=True)

df.drop(columns=['Dt Data Sinistro'], inplace=True)

#### Verificação

In [ ]:
df.head()

In [ ]:
df_clean

In [16]:
X = df_clean

---
## Exploração de dados
### Matriz de Disperção e Correlação
Esse bloco gera uma matriz de gráficos de disperção (*scatter plots*) entre cada par de variáveis numéricas presentes no `df_clean`.<br>
Usamos o pairplot para poder ter uma visualização mais rápida e intuitiva das relações entre as variáveis, facilitando a identificação de correlações que fazem mais sentido.

A segunda célula calcula uma matriz de correlação para identificar o grau de influência entre duas features. 

In [17]:
# Habilitar de acordo com a necessidade removendo o símbolo: #

# Verificar a relação entre as variáveis
# sns.pairplot(df_clean) 

In [18]:
# Habilitar de acordo com a necessidade removendo o símbolo: #

# Calcular a matriz de correlação
# sns.heatmap(df_clean.corr(), annot=True, cmap="coolwarm", linewidths=0.5) 

# plt.show()

---
## Modelo DBSCAN
Este código executa o algoritmo de clustering DBSCAN utilizando os parâmetros previamente definidos, com `eps` igual a 7 e `min_samples` igual a 2. Após a execução do algoritmo, os rótulos resultantes são obtidos e os ruídos, representados por -1, são excluídos para calcular o número de clusters.

Se mais de um cluster for encontrado, é calculada a métrica de avaliação Silhouette Score. Essa métrica fornece uma indicação da qualidade do agrupamento.

Os resultados, incluindo o número de clusters encontrados e a respectiva métrica de avaliação, são exibidos após a célula. Caso apenas um cluster seja identificado, uma mensagem alerta que a maioria dos pontos pode ter sido classificada como ruído. Por fim, os rótulos de clustering são exibidos para inspeção.

In [ ]:
# Definindo os parâmetros do DBSCAN
clustering = DBSCAN(eps=7, min_samples=2)
labels = clustering.fit_predict(df_clean)

# Excluindo ruídos (label -1) para contagem de clusters
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

if n_clusters > 1:  # Calcula as métricas apenas se houver mais de 1 cluster
    # Calcula os scores de avaliação
    silhouette = silhouette_score(df_clean, labels)

    # Formata a saída com informações claras
    print(f"O número de clusters encontrados foi: {n_clusters}")
    print(f"Silhouette Score: {silhouette} (quanto maior, melhor [max = 1])")
else:
    print(f"Apenas {n_clusters} cluster(s) foi/foram encontrado(s). DBSCAN pode ter classificado a maioria dos pontos como ruído.")

# Exibe as labels para inspeção
print(f"Labels do clustering: {clustering.labels_}")

---
## Finetunning de Hiperparâmetro (GridSearch)

Este código define uma função para avaliar o algoritmo DBSCAN com diferentes combinações de parâmetros `eps` e `min_samples`. A função utiliza uma barra de progresso para acompanhar o andamento da avaliação em um conjunto de dados limpos. Os melhores parâmetros são atualizados com base no Davies-Bouldin Score, onde valores mais baixos indicam melhores agrupamentos.

Quando ativado, o código executa uma busca em grade (*gird search*) manualmente, testando uma faixa de valores para ambos os parâmetros e exibindo os melhores resultados no final. O melhor resultado obtido foi um valor de 7 para `eps` e 2 para `min_samples`.

### **ATENÇÃO!!!**
**O código está comentado devido ao tempo de execução prolongado (pode chegar a horas dependendo do computador).**

In [20]:
# def evaluate_dbscan(df_clean, eps_values, min_samples_values):
#     best_eps = None
#     best_min_samples = None
#     best_score = np.inf
#     best_n_clusters = 0
    
#     total_iterations = len(eps_values) * len(min_samples_values)
#     progress_bar = tqdm(total=total_iterations, desc="Overall Progress")
    
#     for eps in eps_values:
#         for min_samples in min_samples_values:
#             # Executa o DBSCAN com a combinação atual de parâmetros
#             dbscan = DBSCAN(eps=eps, min_samples=min_samples)
#             labels = dbscan.fit_predict(df_clean)
            
#             # Exclui os pontos rotulados como ruído (-1) para contar o número de clusters
#             n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
            
#             # Apenas considera o resultado se o número de clusters for menor que 7
#             if 2 <= n_clusters < 10000:  # Ajuste para desconsiderar soluções triviais com 1 cluster
#                 # Calcular o Davies-Bouldin Score (menor é melhor)
#                 score = davies_bouldin_score(df_clean, labels)
                
#                 # Se o novo score for melhor, atualizamos os melhores parâmetros
#                 if score < best_score:
#                     best_eps = eps
#                     best_min_samples = min_samples
#                     best_score = score
#                     best_n_clusters = n_clusters
                
#                 # Print current results
#                 print(f"eps: {eps}, min_samples: {min_samples}, score: {score:}, clusters: {n_clusters}")
            
#             progress_bar.update(1)
    
#     progress_bar.close()
#     return best_eps, best_min_samples, best_score, best_n_clusters

# eps_values = np.arange(1, 80, 1)
# min_samples_values = np.arange(2, 50, 1)


# # Executar o grid search manual
# print("Starting DBSCAN evaluation...")
# best_eps, best_min_samples, best_score, best_n_clusters = evaluate_dbscan(df, eps_values, min_samples_values)

# # Exibir os melhores parâmetros e resultados
# print("\nFinal Results:")
# print(f"Melhor eps: {best_eps}")
# print(f"Melhor min_samples: {best_min_samples}")
# print(f"Melhor Davies-Bouldin Score: {best_score/10:}")
# print(f"Número de clusters: {best_n_clusters}")

### Visualização 3D do Clustering com DBSCAN

Este trecho de código realiza a visualização dos resultados do clustering DBSCAN em um gráfico interativo 3D. A redução da dimensionalidade é feita utilizando a técnica de Análise de Componentes Principais (PCA). O PCA é um método que transforma dados de alta dimensão em uma forma mais simples, preservando a maior quantidade possível de informação. Em outras palavras, ele ajuda a resumir dados complexos em menos variáveis (componentes), facilitando a visualização e a interpretação.

Os dados transformados em 3D são então plotados utilizando a biblioteca Plotly, onde cada ponto representa um dado do conjunto original. As cores dos marcadores são determinadas pelos rótulos gerados pelo algoritmo DBSCAN, utilizando uma escala de cores 'Plasma' para uma melhor visualização dos clusters.

O layout do gráfico é configurado com títulos para os eixos e para o gráfico em si, permitindo uma apresentação clara dos resultados. Por fim, o gráfico **interativo** é exibido, permitindo ao usuário **interagir** com a visualização, **girando e ampliando** a visualização conforme necessário.


In [ ]:
# Redução para 3D com PCA
pca_3d = PCA(n_components=3)
X_pca_3d = pca_3d.fit_transform(X)

# Cria o gráfico interativo 3D
fig = go.Figure()

# Adiciona os pontos dos clusters
fig.add_trace(go.Scatter3d(
    x=X_pca_3d[:, 0], 
    y=X_pca_3d[:, 1], 
    z=X_pca_3d[:, 2], 
    mode='markers',
    marker=dict(
        size=5,
        color=clustering.labels_, # Cores baseadas nos clusters
        colorscale='Plasma',      # Escolhe o esquema de cores
        opacity=0.8
    )
))

# Configura o layout
fig.update_layout(
    title="DBSCAN Clustering in 3D with PCA",
    scene=dict(
        xaxis_title="First PC",
        yaxis_title="Second PC",
        zaxis_title="Third PC"
    ),
    width=800,
    height=700
)

# Mostra o gráfico interativo
fig.show()


### Distribuição do Tamanho dos Clusters

Este código conta o número de pontos em cada cluster gerado pelo algoritmo DBSCAN. A função `np.unique` é utilizada para identificar os rótulos únicos dos clusters e suas respectivas contagens. Em seguida, um gráfico de barras é criado para mostrar a distribuição do tamanho de cada cluster. Cada barra representa um cluster, e a altura da barra corresponde ao número de pontos que pertencem a esse cluster.

O gráfico é configurado com títulos e rótulos apropriados para os eixos, permitindo uma interpretação clara da distribuição dos pontos em cada cluster. Por fim, o gráfico é exibido, facilitando a visualização do desempenho do algoritmo de clustering.


In [ ]:
# Contar o número de pontos em cada cluster
unique, counts = np.unique(clustering.labels_, return_counts=True)

# Ordena os clusters e suas contagens
sorted_indices = np.argsort(unique)
sorted_unique = unique[sorted_indices]
sorted_counts = counts[sorted_indices]

# Plot do tamanho de cada cluster
plt.figure(figsize=(10, 6))
# Define a largura das barras
bar_width = 0.9  # Ajuste este valor conforme necessário
plt.bar(sorted_unique, sorted_counts, width=bar_width, color='dodgerblue')
plt.title("Distribuição de Sinistros por Cluster")
plt.xlabel("Cluster")
plt.ylabel("Número de Sinistros")
plt.xticks(sorted_unique)  # Garante que todos os rótulos de cluster sejam mostrados
plt.show()


### Adição de Rótulos de Cluster ao DataFrame

Este trecho de código adiciona os rótulos de cluster obtidos do algoritmo DBSCAN ao DataFrame `df_clean`, criando uma nova coluna chamada 'Cluster'. Em seguida, uma coluna 'Cluster' também é criada no DataFrame original `df`, inicializada com valores NaN.

Os rótulos de cluster são então preenchidos no DataFrame original apenas para as linhas que não foram removidas como outliers. Isso é feito utilizando o índice do DataFrame limpo `df_clean`. Por fim, as linhas do DataFrame original que ainda têm valores NaN na coluna 'Cluster' são removidas, resultando em um DataFrame que agora inclui a coluna 'Cluster', mas sem as linhas que eram outliers.

O resultado final é exibido, permitindo verificar as primeiras linhas do DataFrame atualizado.


In [ ]:
# Supondo que df seja o DataFrame original e labels seja uma lista ou array com os rótulos de cluster
df['Cluster'] = float('nan')  # Cria a coluna 'Cluster' com NaN

# Adiciona os rótulos de cluster apenas para as linhas que não foram removidas
df.loc[df_clean.index, 'Cluster'] = labels  # Usando .loc para evitar o SettingWithCopyWarning

# Remove as linhas onde 'Cluster' é NaN (ou seja, linhas que eram outliers)
df = df.dropna(subset=['Cluster'])

# Exibe as primeiras linhas do DataFrame atualizado
df.head()


### Análise de Clusters

Este código define uma função chamada `analyze_cluster` que realiza uma análise das estatísticas de interesse para cada cluster presente no DataFrame. As features analisadas incluem 'Codigo Empresa Sinistro', 'Sexo Sinistro', 'Faixa-Etária Nova Sinistro', 'Descricao Plano Sinistro' e 'Valor Pago Sinistro'.

A função começa identificando todos os clusters únicos e calculando a soma total do "Valor Pago Sinistro". Para cada cluster, são exibidas informações como o número de entradas no cluster e, para cada feature:

- **Valor Pago Sinistro**: apresenta a soma absoluta e a soma relativa em relação ao valor total.
- **Demais Features**: calcula a frequência absoluta e percentual dos valores.

O resultado da análise é impresso, com uma separação clara entre os clusters, facilitando a visualização e compreensão dos dados por cluster. A função é então executada com o DataFrame `df`, permitindo uma avaliação detalhada de cada cluster.

#### **ATENÇÃO!!!**
**O código gera um output muito grande, por isso é necessário abri-lo em um formato scrolável ou em um editor de texto, que estão demarcados nas seguintes opções após a exibição parcial do output:**

Output is truncated. View as a ***scrollable*** element or open in a ***text editor***

In [ ]:
# Vamos considerar as features de interesse
features = [
    'Codigo Empresa Sinistro',
    'Sexo Sinistro',
    'Faixa-Etária Nova Sinistro',
    'Descricao Plano Sinistro',
    'Valor Pago Sinistro'
]

# Função para calcular estatísticas por cluster
def analyze_cluster(df, cluster_col='Cluster'):
    clusters = df[cluster_col].unique()
    
    # Calcula a soma total do "Valor Pago Sinistro" em todos os clusters
    total_valor_pago = df['Valor Pago Sinistro'].sum()
    
    for cluster in clusters:
        print(f"\n\n{'='*30}")  # Linha de separação
        print(f"**Cluster {cluster} Analysis:**")  # Título em negrito
        print(f"  ClusterID: {cluster}")  # Indentação
        cluster_data = df[df[cluster_col] == cluster]
        cluster_count = len(cluster_data)
        
        print(f"  Cluster_count: {cluster_count}")
        
        for feature in features:
            if feature == 'Valor Pago Sinistro':
                # Soma absoluta e soma relativa para o Valor Pago Sinistro
                soma_absoluta = cluster_data[feature].sum()
                soma_relativa = (soma_absoluta / total_valor_pago) * 100
                print(f"\n{feature}:")
                print(f"  Soma Absoluta: {soma_absoluta:.2f}")
                print(f"  Soma Relativa ao valor total: {soma_relativa:.2f}%")
            else:
                # Frequência absoluta e percentual para as outras features
                freq_abs = cluster_data[feature].value_counts()
                freq_pct = (freq_abs / cluster_count) * 100
                
                print(f"\n{feature}:")
                for value, count in freq_abs.items():
                    pct = freq_pct[value]
                    print(f"  {value} = {count} ({pct:.1f}%)")

# Executar a análise
analyze_cluster(df)


### Gráfico da Soma do Valor Pago por Cluster

Este código define a função `plot_sum_valor_pago`, que cria um gráfico de barras para visualizar a soma do "Valor Pago Sinistro" por cluster. A função utiliza a biblioteca `matplotlib` para gerar o gráfico.

Primeiro, a função calcula a soma do valor pago por cluster usando o método `groupby` do `pandas`, seguido de um `reset_index` para formatar os dados corretamente. Em seguida, os clusters são ordenados com base no valor da soma em ordem decrescente.

Um gráfico de barras é então gerado, aplicando o colormap `viridis` para colorir as barras. O eixo Y é rotulado com "Soma do Valor Pago Sinistro (R$)", e o título do gráfico é "Soma do Valor Pago Sinistro por Cluster em Reais".

A função adiciona rótulos apenas para os três maiores valores de soma no gráfico, facilitando a identificação dos clusters mais relevantes. Por fim, os ticks do eixo X são configurados para exibir como inteiros e sem rotação, garantindo uma apresentação clara.

A função é chamada com o DataFrame `df` para gerar e exibir o gráfico.


In [ ]:
# Função para criar gráfico da soma do valor pago por cluster
def plot_sum_valor_pago(df, feature='Valor Pago Sinistro', cluster_col='Cluster'):
    # Calcula a soma do valor pago sinistro por cluster
    sum_values = df.groupby(cluster_col)[feature].sum().reset_index()

    # Ordena os clusters pelo valor da soma
    sum_values = sum_values.sort_values(by=feature, ascending=False)

    plt.figure(figsize=(12, 6))
    
    # Aplicar o colormap viridis para as barras
    colors = cm.viridis(np.linspace(0, 1, len(sum_values)))
    bars = plt.bar(sum_values[cluster_col], sum_values[feature], color=colors)
    
    plt.ylabel('Soma do Valor Pago Sinistro (R$)')
    plt.xlabel("Cluster")
    plt.title('Soma do Valor Pago Sinistro por Cluster em Reais')

    # Adicionar rótulos apenas para os três maiores valores
    for bar in bars:
        yval = bar.get_height()
        if bar.get_x() + bar.get_width()/2 in sum_values[cluster_col][:3].values:
            plt.text(bar.get_x() + bar.get_width()/2, yval, f"{yval:,.0f}", va='bottom', ha='center', fontsize=10)

    # Definir ticks no eixo X para exibir como inteiros e sem rotação
    plt.xticks(ticks=sum_values[cluster_col], labels=sum_values[cluster_col].astype(int), rotation=0)

    plt.tight_layout()
    plt.show()

# Chamar a função para gerar o gráfico
plot_sum_valor_pago(df)


### Distribuição de Sexo por Cluster

Este código calcula e plota a frequência percentual da variável "Sexo Sinistro" por cluster, utilizando um gráfico de barras empilhadas para facilitar a visualização das proporções.

Primeiro, a distribuição de frequência é obtida agrupando os dados por "Cluster" e "Sexo Sinistro". O método `size()` conta o número de ocorrências em cada grupo, e `unstack()` transforma a série resultante em um DataFrame, onde as linhas correspondem aos clusters e as colunas representam os sexos. Valores ausentes são preenchidos com zero usando `fillna(0)`.

Em seguida, a porcentagem de cada sexo dentro de cada cluster é calculada dividindo-se o número de ocorrências pelo total de ocorrências em cada cluster, multiplicando por 100 para obter a porcentagem.

Um gráfico de barras empilhadas é então gerado para visualizar a distribuição percentual, utilizando as cores `lightcoral` para feminino e `lightblue` para masculino. O gráfico é rotulado com um título, e os eixos X e Y são configurados para exibir as informações de forma clara.

A legenda é ajustada para identificar claramente os sexos, e linhas de grade horizontais são adicionadas para facilitar a leitura dos dados. Os rótulos do eixo X são formatados para serem inteiros e exibidos em pé.

Por fim, o eixo Y é formatado para exibir os valores como porcentagens, e o gráfico é exibido com o layout ajustado para uma melhor apresentação.


In [ ]:
# Frequência percentual de Sexo Sinistro por Cluster
sexo_dist = df.groupby(['Cluster', 'Sexo Sinistro']).size().unstack().fillna(0)

# Calcular a porcentagem dentro de cada cluster
sexo_dist_percentage = sexo_dist.div(sexo_dist.sum(axis=1), axis=0) * 100

# Plotar o gráfico com as barras empilhadas
sexo_dist_percentage.plot(kind='bar', stacked=True, figsize=(12, 7), color=['lightcoral', 'lightblue'])

# Ajustar o título e rótulos
plt.title('Distribuição de Sexo por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Frequência Percentual')

# Ajustar a legenda para "Masculino" e "Feminino"
plt.legend(['Feminino', 'Masculino'], title='Sexo')

# Adicionar linhas de grade horizontais
plt.grid(axis='y', color="lightgray")

# Ajustar os rótulos do eixo x para ficarem de pé e serem inteiros
plt.xticks(ticks=np.arange(len(sexo_dist_percentage.index)), 
           labels=sexo_dist_percentage.index.astype(int), rotation=0)

# Formatar o eixo y para exibir como porcentagem
plt.gca().yaxis.set_major_formatter(PercentFormatter())

# Exibir o gráfico
plt.tight_layout()
plt.show()


### Distribuição de Faixa Etária por Cluster

Este código calcula e plota a frequência percentual da variável "Faixa-Etária Nova Sinistro" por cluster, utilizando um gráfico de barras empilhadas para ilustrar a proporção de cada faixa etária dentro dos diferentes clusters.

Primeiro, é feita uma cópia do DataFrame `df_clean` e uma nova coluna chamada "Cluster" é adicionada a ele, que contém os rótulos obtidos do modelo de clustering.

Em seguida, a distribuição de frequência é calculada agrupando os dados por "Cluster" e "Faixa-Etária Nova Sinistro". O método `size()` conta o número de ocorrências em cada combinação de cluster e faixa etária, enquanto `unstack(fill_value=0)` transforma a série resultante em um DataFrame onde as linhas correspondem aos clusters e as colunas representam as faixas etárias. O parâmetro `fill_value=0` garante que clusters sem nenhuma ocorrência em uma faixa etária sejam representados com zero.

Para calcular a porcentagem de cada faixa etária dentro de cada cluster, os valores são normalizados dividindo-se o número de ocorrências de cada faixa etária pelo total de ocorrências no respectivo cluster, multiplicando por 100 para obter a porcentagem.

Um gráfico de barras empilhadas é gerado para visualizar a distribuição percentual, utilizando uma paleta de cores do `tab10` do Matplotlib. O gráfico é rotulado com um título, e os eixos X e Y são configurados para apresentar as informações de forma clara.

A legenda é ajustada para exibir as faixas etárias de 0 a 9 e posicionada fora do gráfico para melhor visualização. Linhas de grade horizontais são adicionadas para facilitar a leitura dos dados, e os rótulos do eixo X são formatados para serem inteiros e exibidos na horizontal.

Por fim, o eixo Y é formatado para mostrar os valores como porcentagens, e o gráfico é exibido com o layout ajustado para uma melhor apresentação.

In [ ]:
# Supondo que df_with_labels e labels já estejam definidos como no seu código anterior.

df_with_labels = df_clean.copy()
df_with_labels['Cluster'] = labels  # Adiciona a coluna com os labels  

# Frequência percentual de Faixa Etária por Cluster
faixa_etaria_dist = df_with_labels.groupby(['Cluster', 'Faixa-Etária Nova Sinistro']).size().unstack(fill_value=0)

# Calcular a porcentagem dentro de cada cluster
faixa_etaria_dist_percentage = faixa_etaria_dist.div(faixa_etaria_dist.sum(axis=1), axis=0) * 100

# Garantir que as faixas etárias de 0 a 9 sejam incluídas na legenda
faixa_etaria_labels = np.arange(10)  # Criar uma lista de 0 a 9 para a legenda

# Plotar o gráfico com as barras empilhadas
faixa_etaria_dist_percentage.plot(kind='bar', stacked=True, figsize=(12, 7), color=plt.cm.tab10(np.arange(len(faixa_etaria_dist_percentage.columns))))

# Ajustar o título e rótulos
plt.title('Distribuição de Faixa Etária por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Frequência Percentual')

# Ajustar a legenda para exibir de 0 a 9 e posicioná-la fora do gráfico
plt.legend(faixa_etaria_labels, title='Faixa Etária', loc='upper left', bbox_to_anchor=(1, 1))

# Adicionar linhas de grade horizontais
plt.grid(axis='y', color="lightgray")

# Ajustar os rótulos do eixo x para ficarem de pé e serem inteiros
plt.xticks(ticks=np.arange(len(faixa_etaria_dist_percentage.index)),
           labels=faixa_etaria_dist_percentage.index.astype(int), rotation=0)

# Formatar o eixo y para exibir como porcentagem
plt.gca().yaxis.set_major_formatter(PercentFormatter())

# Exibir o gráfico
plt.tight_layout()
plt.show()


Este notebook implementa o algoritmo de clustering DBSCAN (Density-Based Spatial Clustering of Applications with Noise) para analisar um conjunto de dados sobre sinistros. O processo começa com a definição dos parâmetros do DBSCAN, como `eps` e `min_samples`, que são ajustados para otimizar a formação de clusters. Após a execução do algoritmo, são realizadas análises estatísticas e visuais dos resultados, incluindo a contagem de pontos em cada cluster, a soma do valor pago por sinistros e a distribuição de variáveis categóricas, como sexo e faixa etária, entre os clusters formados.

A análise detalhada dos resultados permite a identificação de padrões e comportamentos dentro dos grupos, contribuindo para uma melhor compreensão dos dados. Essa abordagem valida a eficácia do DBSCAN na identificação de estruturas densas nos dados e fornece insights práticos que podem ser utilizados em estratégias de gerenciamento de sinistros, desenvolvimento de políticas e aprimoramento dos serviços oferecidos pela Unipar.


---
# Cálculo de métricas

Aqui é realizado o cálculo das métricas escolhidas: Silhouette Score, Davis-Bouldin Index e Calinski-Harabasz Index. Elas ajudam a verificar se a formação dos *clusters* foi bem-sucedida, com base na qualidade da segmentação obtida. O resultado dessas métricas sozinho não deve ser a única maneira de avaliar se o modelo de fato é o melhor, estamos levando em consideração como os *clusters* foram segmentados pelo algoritmo para ter certeza de que as divisões feitas realmente fazem sentido para o projeto.

- O Davies-Bouldin Score deve ser minimizado (mínimo possível é 0).
- O Silhouette Score deve ser maximizado (máximo possível é 1).
- O Calinski-Harabasz Score também deve ser maximizado (não há um limite superior).


In [ ]:
# Definindo os parâmetros do DBSCAN
clustering = DBSCAN(eps=7, min_samples=2)
labels = clustering.fit_predict(df_clean)

# Excluindo ruídos (label -1) para contagem de clusters
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

if n_clusters > 1:  # Calcula as métricas apenas se houver mais de 1 cluster
    # Calcula os scores de avaliação
    db_score = davies_bouldin_score(df_clean, labels)
    silhouette = silhouette_score(df_clean, labels)
    calinski = calinski_harabasz_score(df_clean, labels)

    # Formata a saída com informações claras
    print(f"Davies-Bouldin Score: {db_score} (quanto menor, melhor [min = 0])")
    print(f"Silhouette Score: {silhouette} (quanto maior, melhor [max = 1])")
    print(f"Calinski-Harabasz Score: {calinski} (quanto maior, melhor [max = inf])")
else:
    print(f"Apenas {n_clusters} cluster(s) foi/foram encontrado(s). DBSCAN pode ter classificado a maioria dos pontos como ruído.")
